# Building a Database for Crime Reports in Boston

In this project, we will create a PostgreSQL database named `crimes_db` to store and manage crime data from Boston. Using the provided `boston.csv` dataset, we will:

- Design a schema (`crimes`) and a table (`boston_crimes`) with appropriate datatypes.
- Import the data into the database.
- Create user roles (`readonly` and `readwrite`) with appropriate privileges.
- Assign users to these roles to manage database access.